# IntentResolution, TaskAdherence and Relevance Evaluators

## Objective
This sample demonstrates to how to use intent resolution, task adherence and relevance evaluators on agent data. The supported input formats include:
- simple data such as strings;
- user-agent conversations in the form of list of agent messages. 

## Time

You should expect to spend about 30 minutes running this notebook. 

## Before you begin
For quality evaluation, you need to deploy a `gpt` model supporting JSON mode. We recommend a model `gpt-4o` or `gpt-4o-mini` for their strong reasoning capabilities.    

### Prerequisite
```bash
pip install azure-ai-projects azure-identity azure-ai-evaluation
```
Set these environment variables with your own values:
1) **PROJECT_CONNECTION_STRING** - The project connection string, as found in the overview page of your Azure AI Foundry project.
2) **MODEL_DEPLOYMENT_NAME** - The deployment name of the model for this AI-assisted evaluator, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.
3) **AZURE_OPENAI_ENDPOINT** - Azure Open AI Endpoint to be used for evaluation.
4) **AZURE_OPENAI_API_KEY** - Azure Open AI Key to be used for evaluation.
5) **AZURE_OPENAI_API_VERSION** - Azure Open AI Api version to be used for evaluation.
6) **AZURE_SUBSCRIPTION_ID** - Azure Subscription Id of Azure AI Project
7) **PROJECT_NAME** - Azure AI Project Name
8) **RESOURCE_GROUP_NAME** - Azure AI Project Resource Group Name


The Intent Resolution evaluator measures how well an agent has identified and resolved the user intent.
The scoring is on a 1-5 integer scale and is as follows:

  - Score 1: Response completely unrelated to user intent
  - Score 2: Response minimally relates to user intent
  - Score 3: Response partially addresses the user intent but lacks complete details
  - Score 4: Response addresses the user intent with moderate accuracy but has minor inaccuracies or omissions
  - Score 5: Response directly addresses the user intent and fully resolves it

The evaluation requires the following inputs:

  - Query    : The user query. Either a string with a user request or a list of messages with previous requests from the user and responses from the assistant, potentially including a system message.
  - Response : The response to be evaluated. Either a string or a message with the response from the agent to the last user query.

There is a third optional parameter:
  - ToolDefinitions : The list of tool definitions the agent can call. This may be useful for the evaluator to better assess if the right tool was called to resolve a given intent.

### Initialize Intent Resolution Evaluator


In [ ]:
import os
from azure.ai.evaluation import AzureOpenAIModelConfiguration
from azure.ai.evaluation import IntentResolutionEvaluator, TaskAdherenceEvaluator, RelevanceEvaluator
from pprint import pprint

from dotenv import load_dotenv

# Load environment variables from .env file. Please provide your .env file
load_dotenv(override=True)

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["MODEL_DEPLOYMENT_NAME"],
)

intent_resolution_evaluator = IntentResolutionEvaluator(model_config)
task_adherence_evaluator = TaskAdherenceEvaluator(model_config)
relevance_evaluator = RelevanceEvaluator(model_config)

### Samples

#### Evaluating query and response as string

In [ ]:
# Success example. Intent is identified and understood and the response correctly resolves user intent
#Choose the evalfunc you want to test
evalfunc = relevance_evaluator
evalfunc = task_adherence_evaluator
evalfunc = intent_resolution_evaluator

result = evalfunc(
    query="What are the opening hours of the Eiffel Tower?",
    response="Opening hours of the Eiffel Tower are 9:00 AM to 11:00 PM.",
)
print(result)

In [ ]:
# Failure example. Even though intent is correctly identified, the response does not resolve the user intent
result = evalfunc(
    query="What is the opening hours of the Eiffel Tower?",
    response="Please check the official website for the up-to-date information on Eiffel Tower opening hours.",
)
print(result)

#### Evaluating query and response as list of messages

In [ ]:
query = [
    {"role": "system", "content": "You are a friendly and helpful customer service agent."},
    {
        "createdAt": "2025-03-14T06:14:20Z",
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Hi, I need help with the last 2 orders on my account #888. Could you please update me on their status?",
            }
        ],
    },
]

response = [
    {
        "createdAt": "2025-03-14T06:14:30Z",
        "run_id": "0",
        "role": "assistant",
        "content": [{"type": "text", "text": "Hello! Let me quickly look up your account details."}],
    },
    {
        "createdAt": "2025-03-14T06:14:35Z",
        "run_id": "0",
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "tool_call_20250310_001",
                "name": "get_orders",
                "arguments": {"account_number": "888"},
            }
        ],
    },
    {
        "createdAt": "2025-03-14T06:14:40Z",
        "run_id": "0",
        "tool_call_id": "tool_call_20250310_001",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": '[{ "order_id": "123" }, { "order_id": "124" }]'}],
    },
    {
        "createdAt": "2025-03-14T06:14:45Z",
        "run_id": "0",
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "Thanks for your patience. I see two orders on your account. Let me fetch the details for both.",
            }
        ],
    },
    {
        "createdAt": "2025-03-14T06:14:50Z",
        "run_id": "0",
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "tool_call_20250310_002",
                "name": "get_order",
                "arguments": {"order_id": "123"},
            },
            {
                "type": "tool_call",
                "tool_call_id": "tool_call_20250310_003",
                "name": "get_order",
                "arguments": {"order_id": "124"},
            },
        ],
    },
    {
        "createdAt": "2025-03-14T06:14:55Z",
        "run_id": "0",
        "tool_call_id": "tool_call_20250310_002",
        "role": "tool",
        "content": [
            {
                "type": "tool_result",
                "tool_result": '{ "order": { "id": "123", "status": "shipped", "delivery_date": "2025-03-15" } }',
            }
        ],
    },
    {
        "createdAt": "2025-03-14T06:15:00Z",
        "run_id": "0",
        "tool_call_id": "tool_call_20250310_003",
        "role": "tool",
        "content": [
            {
                "type": "tool_result",
                "tool_result": '{ "order": { "id": "124", "status": "delayed", "expected_delivery": "2025-03-20" } }',
            }
        ],
    },
    {
        "createdAt": "2025-03-14T06:15:05Z",
        "run_id": "0",
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "The order with ID 123 has been shipped and is expected to be delivered on March 15, 2025. However, the order with ID 124 is delayed and should now arrive by March 20, 2025. Is there anything else I can help you with?",
            }
        ],
    },
]

# please note that the tool definitions are not strictly required, and that some of the tools below are not used in the example above and that is ok.
# if context length is a concern you can remove the unused tool definitions or even the tool definitions altogether as the impact to the intent resolution evaluation is usual minimal.
tool_definitions = [
    {
        "name": "get_orders",
        "description": "Get the list of orders for a given account number.",
        "parameters": {
            "type": "object",
            "properties": {
                "account_number": {"type": "string", "description": "The account number to get the orders for."}
            },
        },
    },
    {
        "name": "get_order",
        "description": "Get the details of a specific order.",
        "parameters": {
            "type": "object",
            "properties": {"order_id": {"type": "string", "description": "The order ID to get the details for."}},
        },
    },
    {
        "name": "initiate_return",
        "description": "Initiate the return process for an order.",
        "parameters": {
            "type": "object",
            "properties": {"order_id": {"type": "string", "description": "The order ID for the return process."}},
        },
    },
    {
        "name": "update_shipping_address",
        "description": "Update the shipping address for a given account.",
        "parameters": {
            "type": "object",
            "properties": {
                "account_number": {"type": "string", "description": "The account number to update."},
                "new_address": {"type": "string", "description": "The new shipping address."},
            },
        },
    },
]

result = evalfunc(
    query=query,
    response=response,
    tool_definitions=tool_definitions,
)
print(result)

## Batch evaluate and visualize results on Azure AI Foundry
Batch evaluate to leverage asynchronous evaluation on a dataset. 

Optionally, you can go to AI Foundry URL for rich Azure AI Foundry data visualization. You can inspect the evaluation scores and reasoning to quickly identify bugs and issues of your agent to fix and improve. Make sure to authenticate to Azure using `az login` in your terminal before running this cell.


In [ ]:
from azure.ai.evaluation import evaluate

# This sample files contains the evaluation data in JSONL format. Where each line is a run from agent.
# This was saved using agent thread and converter.
file_name = "MSAI_shopping_agent.s100.jsonl"

response = evaluate(
    data=file_name,
    evaluation_name=f"Intent ResolutionV2 Evaluation of MSAI Shopping Agent, 100 sampled conversations",
    evaluators={
        "intent_resolution": intent_resolution_evaluator,
        "task_adherence" : task_adherence_evaluator,
        #"relevance": relevance_evaluator,
    },
    azure_ai_project={
        "subscription_id": os.environ["AZURE_SUBSCRIPTION_ID"],
        "project_name": os.environ["PROJECT_NAME"],
        "resource_group_name": os.environ["RESOURCE_GROUP_NAME"],
    },
)
pprint(f'AI Foundry URL: {response.get("studio_url")}')